In [ ]:
"""
Google Colab Gemini Chatbot
===========================
A secure, feature-rich AI chatbot using Google's Gemini AI.

Features:
- Secure API key management
- Multiple personality modes
- Interactive chat with history
- Comprehensive error handling
- Easy setup and troubleshooting

Author: [Elias Silomba]
License: MIT
"""

In [ ]:
# Google Colab Chatbot using Gemini AI with Environment Variables
# Run this code in Google Colab

!pip install -q google-generativeai python-dotenv

import google.generativeai as genai
import os
from dotenv import load_dotenv
from IPython.display import display, Markdown, clear_output
import time

def create_env_file():
    """Helper function to create .env file with API key"""
    print("🔧 Creating .env file...")
    api_key = input("Enter your Google AI Studio API Key: ").strip()

    if api_key:
        with open('.env', 'w') as f:
            f.write(f"GOOGLE_AI_API_KEY={api_key}\n")
        print("✅ .env file created successfully!")
        print("🔄 Now loading the environment variables...")
        load_dotenv()
        return True
    else:
        print("❌ No API key provided. .env file not created.")
        return False

def check_env_setup():
    """Check if .env file is properly set up"""
    if os.path.exists('.env'):
        load_dotenv()
        api_key = os.getenv('GOOGLE_AI_API_KEY')
        if api_key:
            print("✅ .env file found and API key loaded!")
            return True, api_key
        else:
            print("❌ .env file found but GOOGLE_AI_API_KEY not set properly")
            return False, None
    else:
        print("❌ .env file not found")
        print("You can create one by running: create_env_file()")
        return False, None

def setup_api():
    """Setup the API with proper error handling"""
    is_setup, api_key = check_env_setup()

    if not is_setup:
        print("\n🔧 Let's set up your environment file...")
        choice = input("Would you like to create a .env file now? (y/n): ").strip().lower()

        if choice in ['y', 'yes']:
            if create_env_file():
                is_setup, api_key = check_env_setup()
            else:
                return False
        else:
            print("❌ Cannot proceed without API key. Please set up .env file manually.")
            return False

    if is_setup and api_key:
        try:
            genai.configure(api_key=api_key)
            print("✅ Google AI configured successfully!")
            return True
        except Exception as e:
            print(f"❌ Error configuring Google AI: {e}")
            return False

    return False

print("🚀 Setting up Google AI API...")
api_ready = setup_api()

if not api_ready:
    print("\n❌ Setup failed. Please check your API key and try again.")
else:
    print("✅ Ready to use the chatbot!")

if api_ready:
    try:
        model = genai.GenerativeModel('gemini-1.5-flash')
        print("✅ Using Gemini 1.5 Flash model")
    except Exception as e:
        try:
            model = genai.GenerativeModel('gemini-1.5-pro')
            print("✅ Using Gemini 1.5 Pro model")
        except Exception as e2:
            try:
                model = genai.GenerativeModel('gemini-pro')
                print("✅ Using Gemini Pro model")
            except Exception as e3:
                print(f"❌ Error initializing model: {e3}")
                print("Let's check available models...")
                try:
                    models = genai.list_models()
                    print("Available models:")
                    for m in models:
                        if 'generateContent' in m.supported_generation_methods:
                            print(f"  - {m.name}")
                except Exception as e4:
                    print(f"❌ Could not list models: {e4}")
                api_ready = False

class GeminiChatbot:
    def __init__(self):
        if not api_ready:
            raise Exception("API not configured. Please run setup_api() first.")
        self.chat_history = []
        self.chat = model.start_chat(history=[])

    def get_response(self, user_input):
        """Get response from Gemini AI"""
        try:
            response = self.chat.send_message(user_input)
            return response.text
        except Exception as e:
            return f"Error: {str(e)}"

    def add_to_history(self, user_input, bot_response):
        """Add conversation to history"""
        self.chat_history.append({
            'user': user_input,
            'bot': bot_response,
            'timestamp': time.strftime("%H:%M:%S")
        })

    def display_chat_history(self):
        """Display the chat history in a nice format"""
        print("=== Chat History ===")
        for i, chat in enumerate(self.chat_history, 1):
            print(f"\n[{chat['timestamp']}] Chat {i}")
            print(f"You: {chat['user']}")
            print(f"Bot: {chat['bot']}")
            print("-" * 50)

def run_chatbot():
    """Run the interactive chatbot"""
    if not api_ready:
        print("❌ API not ready. Please run setup_api() first.")
        return

    try:
        chatbot = GeminiChatbot()
    except Exception as e:
        print(f"❌ Failed to initialize chatbot: {e}")
        return

    print("🤖 Gemini Chatbot Started!")
    print("Type 'quit', 'exit', or 'bye' to end the conversation")
    print("Type 'history' to see chat history")
    print("Type 'clear' to clear the screen")
    print("=" * 50)

    while True:
        user_input = input("\nYou: ").strip()

        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("👋 Goodbye! Thanks for chatting!")
            break

        if user_input.lower() == 'history':
            chatbot.display_chat_history()
            continue

        if user_input.lower() == 'clear':
            clear_output()
            print("🤖 Gemini Chatbot - Screen Cleared!")
            print("Type 'quit', 'exit', or 'bye' to end the conversation")
            print("=" * 50)
            continue

        if not user_input:
            continue

        print("Bot: ", end="", flush=True)

        typing_chars = "|/-\\"
        for i in range(8):
            print(f"\rBot: {typing_chars[i % 4]} Thinking...", end="", flush=True)
            time.sleep(0.1)
        print("\r", end="")

        bot_response = chatbot.get_response(user_input)
        print(f"Bot: {bot_response}")

        chatbot.add_to_history(user_input, bot_response)

class CustomChatbot(GeminiChatbot):
    def __init__(self, personality=""):
        super().__init__()
        self.personality = personality
        if personality:
            initial_prompt = f"You are a chatbot with the following personality: {personality}. Please respond to all messages while maintaining this personality."
            self.chat.send_message(initial_prompt)

    def set_personality(self, personality):
        """Change the chatbot's personality"""
        self.personality = personality
        self.chat = model.start_chat(history=[])
        if personality:
            initial_prompt = f"You are a chatbot with the following personality: {personality}. Please respond to all messages while maintaining this personality."
            self.chat.send_message(initial_prompt)
        print(f"✅ Personality updated: {personality}")

def run_enhanced_chatbot():
    """Run enhanced chatbot with additional features"""
    if not api_ready:
        print("❌ API not ready. Please run setup_api() first.")
        return

    print("🚀 Enhanced Gemini Chatbot!")
    print("\nAvailable commands:")
    print("• 'quit', 'exit', 'bye' - End conversation")
    print("• 'history' - Show chat history")
    print("• 'clear' - Clear screen")
    print("• 'personality [description]' - Set bot personality")
    print("• 'help' - Show this help message")
    print("=" * 50)

    personality_choice = input("Set initial personality (or press Enter for default): ").strip()

    try:
        chatbot = CustomChatbot(personality_choice)
    except Exception as e:
        print(f"❌ Failed to initialize chatbot: {e}")
        return

    if personality_choice:
        print(f"✅ Chatbot personality set to: {personality_choice}")

    while True:
        user_input = input("\nYou: ").strip()

        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("👋 Goodbye! Thanks for chatting!")
            break

        if user_input.lower() == 'history':
            chatbot.display_chat_history()
            continue

        if user_input.lower() == 'clear':
            clear_output()
            print("🚀 Enhanced Gemini Chatbot - Screen Cleared!")
            continue

        if user_input.lower() == 'help':
            print("\nAvailable commands:")
            print("• 'quit', 'exit', 'bye' - End conversation")
            print("• 'history' - Show chat history")
            print("• 'clear' - Clear screen")
            print("• 'personality [description]' - Set bot personality")
            print("• 'help' - Show this help message")
            continue

        if user_input.lower().startswith('personality '):
            new_personality = user_input[12:]
            chatbot.set_personality(new_personality)
            continue

        if not user_input:
            continue

        bot_response = chatbot.get_response(user_input)
        print(f"Bot: {bot_response}")

        chatbot.add_to_history(user_input, bot_response)

def ask_gemini(question):
    """Simple function to ask a single question"""
    if not api_ready:
        return "❌ API not ready. Please run setup_api() first."

    try:
        response = model.generate_content(question)
        return response.text
    except Exception as e:
        return f"Error: {str(e)}"

def diagnose_api_issue():
    """Diagnose API connection and key issues"""
    print("🔍 Diagnosing API issues...")

    api_key = os.getenv('GOOGLE_AI_API_KEY')
    if not api_key:
        print("❌ No API key found in environment")
        return False

    print(f"✅ API key found: {api_key[:10]}...{api_key[-5:]}")

    if not api_key.startswith('AIza'):
        print("⚠️  API key doesn't start with 'AIza' - this might be incorrect")
        print("   Google AI Studio API keys usually start with 'AIza'")

    try:
        genai.configure(api_key=api_key)
        print("✅ API configured successfully")
    except Exception as e:
        print(f"❌ API configuration failed: {e}")
        return False

    try:
        print("🧪 Testing model creation...")
        test_model = genai.GenerativeModel('gemini-1.5-flash')
        print("✅ Model creation successful")
        return True
    except Exception as e:
        print(f"❌ Model creation failed: {e}")
        if "404" in str(e):
            print("   This suggests the model name is not available")
        elif "403" in str(e):
            print("   This suggests API key permission issues")
        elif "500" in str(e):
            print("   This suggests server-side issues")
        return False

def test_simple_request():
    """Test a simple API request without listing models"""
    print("🧪 Testing simple API request...")

    api_key = os.getenv('GOOGLE_AI_API_KEY')
    if not api_key:
        print("❌ No API key found")
        return False

    models_to_try = [
        'gemini-1.5-flash',
        'gemini-1.5-pro',
        'gemini-1.0-pro-latest',
        'gemini-1.0-pro',
        'gemini-pro'
    ]

    for model_name in models_to_try:
        try:
            print(f"   Testing {model_name}...")
            model = genai.GenerativeModel(model_name)
            response = model.generate_content("Hello")
            print(f"   ✅ {model_name} works! Response: {response.text[:30]}...")

            globals()['model'] = model
            print(f"   ✅ Global model set to: {model_name}")
            return True

        except Exception as e:
            error_msg = str(e)
            if "404" in error_msg:
                print(f"   ❌ {model_name}: Model not found")
            elif "403" in error_msg:
                print(f"   ❌ {model_name}: Permission denied")
            elif "500" in error_msg:
                print(f"   ❌ {model_name}: Server error")
            else:
                print(f"   ❌ {model_name}: {error_msg[:50]}...")
            continue

    print("❌ No working models found")
    return False

def check_api_key_validity():
    """Check if the API key is valid format and potentially working"""
    api_key = os.getenv('GOOGLE_AI_API_KEY')

    if not api_key:
        print("❌ No API key found")
        return False

    print("🔍 Checking API key...")
    print(f"   Format: {api_key[:6]}...{api_key[-4:]}")
    print(f"   Length: {len(api_key)} characters")

    if len(api_key) < 30:
        print("⚠️  API key seems too short")
    if len(api_key) > 50:
        print("⚠️  API key seems too long")
    if not api_key.startswith('AIza'):
        print("⚠️  API key should start with 'AIza'")
        print("   Make sure you're using Google AI Studio API key, not other Google API keys")

    if ' ' in api_key:
        print("❌ API key contains spaces - this will cause issues")
    if api_key != api_key.strip():
        print("❌ API key has leading/trailing whitespace")

    return True

def fix_common_issues():
    """Guide user through fixing common issues"""
    print("🔧 Common Issues and Solutions:")
    print("\n1. API Key Issues:")
    print("   • Make sure you're using Google AI Studio API key (https://makersuite.google.com/app/apikey)")
    print("   • API key should start with 'AIza'")
    print("   • Don't use Google Cloud or other Google API keys")
    print("\n2. Network Issues:")
    print("   • Check your internet connection")
    print("   • Try restarting Colab runtime")
    print("\n3. Model Availability:")
    print("   • Some models might not be available in your region")
    print("   • Try different model names")
    print("\n4. Steps to fix:")
    print("   a. Run: diagnose_api_issue()")
    print("   b. Run: check_api_key_validity()")
    print("   c. Run: test_simple_request()")
    print("   d. If still failing, create a new API key")

def recreate_env_file():
    """Help user recreate .env file with proper API key"""
    print("🔧 Let's recreate your .env file...")
    print("\n1. Go to https://makersuite.google.com/app/apikey")
    print("2. Create a new API key (or copy existing one)")
    print("3. Make sure it starts with 'AIza'")

    api_key = input("\nEnter your Google AI Studio API Key: ").strip()

    if api_key:
        if not api_key.startswith('AIza'):
            print("⚠️  Warning: API key doesn't start with 'AIza'")
            proceed = input("Continue anyway? (y/n): ").strip().lower()
            if proceed not in ['y', 'yes']:
                print("❌ Cancelled")
                return False

        with open('.env', 'w') as f:
            f.write(f"GOOGLE_AI_API_KEY={api_key}\n")

        print("✅ .env file updated!")

        load_dotenv(override=True)

        print("🧪 Testing new API key...")
        genai.configure(api_key=api_key)

        return test_simple_request()
    else:
        print("❌ No API key provided")
        return False

def list_available_models():
    """List all available models and their capabilities"""
    if not api_ready:
        print("❌ API not ready. Please run setup_api() first.")
        return

    try:
        print("🔍 Checking available models...")
        models = genai.list_models()

        print("\n📋 Available models for content generation:")
        generation_models = []

        for m in models:
            if 'generateContent' in m.supported_generation_methods:
                model_name = m.name.split('/')[-1]
                generation_models.append(model_name)
                print(f"  ✅ {model_name}")

        if generation_models:
            print(f"\n💡 Recommended: Use one of these models")
            print("   Most common: gemini-1.5-flash, gemini-1.5-pro")
        else:
            print("❌ No models found for content generation")

        return generation_models

    except Exception as e:
        print(f"❌ Error listing models: {e}")
        return []

def test_model(model_name=None):
    """Test a specific model or find a working one"""
    if not api_ready:
        print("❌ API not ready. Please run setup_api() first.")
        return False

    test_models = []
    if model_name:
        test_models = [model_name]
    else:
        test_models = [
            'gemini-1.5-flash',
            'gemini-1.5-pro',
            'gemini-1.0-pro',
            'gemini-pro'
        ]

    for model_name in test_models:
        try:
            print(f"🧪 Testing model: {model_name}")
            test_model = genai.GenerativeModel(model_name)
            response = test_model.generate_content("Say hello")
            print(f"✅ {model_name} works! Response: {response.text[:50]}...")

            global model
            model = test_model
            print(f"✅ Model updated to: {model_name}")
            return True

        except Exception as e:
            print(f"❌ {model_name} failed: {str(e)[:100]}...")
            continue

    print("❌ No working models found")
    return False

def show_instructions():
    """Show usage instructions"""
    print("🎯 How to use this chatbot:")
    print("\n1. Get your API key:")
    print("   - Go to https://makersuite.google.com/app/apikey")
    print("   - Create a new API key")
    print("   - Copy the API key")
    print("\n2. Setup (choose one method):")
    print("   Method A - Let the code create .env file:")
    print("   • Just run the code, it will prompt you for the API key")
    print("\n   Method B - Create .env file manually:")
    print("   • Create a file named '.env' (with the dot)")
    print("   • Add this line: GOOGLE_AI_API_KEY=your_actual_api_key_here")
    print("   • Upload to Colab using the Files panel")
    print("\n3. If you get model errors:")
    print("   - Run: list_available_models() - to see what's available")
    print("   - Run: test_model() - to find a working model")
    print("   - Run: test_model('model-name') - to test a specific model")
    print("\n4. Run the chatbot:")
    print("   - For basic chatbot: run_chatbot()")
    print("   - For enhanced chatbot: run_enhanced_chatbot()")
    print("   - For single questions: ask_gemini('your question')")
    print("\n5. If you need to reconfigure:")
    print("   - Run: setup_api()")
    print("   - Or run: create_env_file()")
    print("\n6. Example usage:")
    print("   # Check available models")
    print("   list_available_models()")
    print("\n   # Test and set working model")
    print("   test_model()")
    print("\n   # Single question")
    print("   print(ask_gemini('What is machine learning?'))")
    print("\n   # Start interactive chat")
    print("   run_enhanced_chatbot()")

print("\n" + "="*60)
show_instructions()
print("="*60)

if api_ready:
    print("\n✅ Everything is ready! You can now use:")
    print("• run_chatbot() - Basic chatbot")
    print("• run_enhanced_chatbot() - Enhanced chatbot")
    print("• ask_gemini('your question') - Single question")
    print("• list_available_models() - Check available models")
    print("• test_model() - Find working model")
else:
    print("\n🔧 Setup needed. The code will guide you through it.")
    print("If you missed the setup, run: setup_api()")

print("\n🔍 If you got a model error (404), run these commands:")
print("1. list_available_models() - to see available models")
print("2. test_model() - to find and set a working model")

# Quick start examples (uncomment to use):
# run_enhanced_chatbot()        # Start enhanced chatbot
# print(ask_gemini("Hello!"))   # Ask a single question

In [ ]:
# SECURITY SETUP CODE - Copy this entire block into a new cell in your notebook

print("🔧 Setting up GitHub security files...")

gitignore_content = """# Environment Variables and API Keys
.env
.env.local
.env.production
.env.staging
*.env

# API Keys (additional protection)
*api_key*
*API_KEY*
secrets.json
config.json

# Google Colab Files
.ipynb_checkpoints/
*.ipynb_checkpoints

# Python
__pycache__/
*.py[cod]
*$py.class
*.so
.Python
build/
develop-eggs/
dist/
downloads/
eggs/
.eggs/
lib/
lib64/
parts/
sdist/
var/
wheels/
*.egg-info/
.installed.cfg
*.egg

# Virtual Environment
venv/
env/
ENV/
.venv/

# IDE Files
.vscode/
.idea/
*.swp
*.swo
*~

# OS Files
.DS_Store
.DS_Store?
._*
.Spotlight-V100
.Trashes
ehthumbs.db
Thumbs.db

# Logs
*.log
logs/

# Temporary files
*.tmp
*.temp
"""

with open('.gitignore', 'w') as f:
    f.write(gitignore_content)
print("✅ .gitignore file created")

env_example_content = """# Google AI Studio API Key Configuration
# Get your API key from: https://makersuite.google.com/app/apikey
GOOGLE_AI_API_KEY=your_google_ai_studio_api_key_here

# Instructions:
# 1. Copy this file to .env (cp .env.example .env)
# 2. Replace 'your_google_ai_studio_api_key_here' with your actual API key
# 3. Your API key should start with 'AIza'
# 4. Never commit the .env file to version control
"""

with open('.env.example', 'w') as f:
    f.write(env_example_content)
print("✅ .env.example file created")

readme_content = """# Google Colab Gemini Chatbot

A powerful AI chatbot built with Google's Gemini AI, designed to run in Google Colab with secure API key management.

## Features

- 🤖 Interactive chatbot with conversation memory
- 🎭 Customizable AI personalities
- 📜 Chat history tracking
- 🔧 Multiple chatbot modes (basic and enhanced)
- 🛡️ Secure API key management
- ⚡ Easy setup in Google Colab

## Quick Start

- Go to [Google AI Studio](https://makersuite.google.com/app/apikey)
- Create a new API key
- Copy the API key (starts with 'AIza')


1. Open [Google Colab](https://colab.research.google.com/)
2. Upload the notebook file
3. Run the code - it will guide you through setup


The code will automatically prompt you to enter your API key and create the necessary configuration files.

## Available Functions

### Basic Usage
```python
# Single question
print(ask_gemini("What is machine learning?"))

# Start basic chatbot
run_chatbot()

# Start enhanced chatbot with personalities
run_enhanced_chatbot()
```

### Troubleshooting
```python
# Check API setup
diagnose_api_issue()

# Test models
test_simple_request()

# Check available models
list_available_models()
```

## Chatbot Commands

While chatting, you can use these commands:
- `quit`, `exit`, `bye` - End conversation
- `history` - Show chat history
- `clear` - Clear screen
- `personality [description]` - Set AI personality
- `help` - Show help message

## Security

This project uses environment variables to keep your API key secure:
- API keys are stored in `.env` file (not tracked by Git)
- Never hardcode API keys in source code
- `.gitignore` prevents accidental key exposure

## Requirements

- Google Colab account
- Google AI Studio API key
- Internet connection

## Troubleshooting

### Common Issues:
1. **API Key Errors**: Make sure your key starts with 'AIza'
2. **Model Not Found**: Run `test_simple_request()` to find working models
3. **500 Errors**: Usually temporary, try again or use different model

### Getting Help:
- Run `fix_common_issues()` for guided troubleshooting
- Check that your API key is from Google AI Studio (not Google Cloud)

## Contributing

1. Fork the repository
2. Create your feature branch
3. Make sure to test with your own API key
4. Submit a pull request

## License

MIT License - feel free to use and modify!

## Disclaimer

This project is for educational purposes. Keep your API keys secure and monitor your usage to avoid unexpected charges.
"""

with open('README.md', 'w') as f:
    f.write(readme_content)
print("✅ README.md file created")

def verify_security_setup():
    """Verify that all security files are properly created"""
    import os

    files_to_check = ['.gitignore', '.env.example', 'README.md']

    print("\n🔍 Security Setup Verification:")
    all_good = True

    for file in files_to_check:
        if os.path.exists(file):
            print(f"✅ {file} - Created successfully")
        else:
            print(f"❌ {file} - Missing!")
            all_good = False

    # Check if .env is protected in .gitignore
    if os.path.exists('.gitignore'):
        with open('.gitignore', 'r') as f:
            content = f.read()
            if '.env' in content:
                print("✅ .env protection - Properly configured in .gitignore")
            else:
                print("❌ .env protection - Not found in .gitignore")
                all_good = False

    # Check current .env file status
    if os.path.exists('.env'):
        print("⚠️  .env file exists - Make sure this is in .gitignore!")
    else:
        print("✅ No .env file found - This is good for GitHub")

    print(f"\n{'🎉' if all_good else '⚠️ '} Security setup {'complete' if all_good else 'needs attention'}!")

    if all_good:
        print("\n📋 Next steps for GitHub:")
        print("1. Download these files from Colab (see instructions below)")
        print("2. Add them to your local project")
        print("3. Copy .env.example to .env locally")
        print("4. Add your API key to .env")
        print("5. Commit and push to GitHub")

    return all_good

def show_download_instructions():
    """Show how to download files from Colab"""
    print("\n📥 How to download files from Google Colab:")
    print("\nMethod 1 - Manual Download:")
    print("1. In the left sidebar, click the folder icon 📁")
    print("2. Right-click on each file (.gitignore, .env.example, README.md)")
    print("3. Select 'Download' from the menu")
    print("4. Save them to your local project folder")

    print("\nMethod 2 - Programmatic Download:")
    print("Run these commands in a new cell:")
    print("from google.colab import files")
    print("files.download('.gitignore')")
    print("files.download('.env.example')")
    print("files.download('README.md')")


def download_security_files():
    """Download all security files"""
    try:
        from google.colab import files
        print("📥 Downloading security files...")
        files.download('.gitignore')
        files.download('.env.example')
        files.download('README.md')
        print("✅ All files downloaded successfully!")
    except Exception as e:
        print(f"❌ Download failed: {e}")
        print("Try manual download from the file browser instead.")

# Run the verification
verify_security_setup()

# Show download instructions
print("\n" + "="*60)
show_download_instructions()

print(f"\n🎯 Quick Commands Available:")
print("verify_security_setup()      # Check if everything is set up")
print("show_download_instructions()  # How to download files")
print("download_security_files()     # Auto-download all files")

print(f"\n{'='*60}")
print("✅ Security setup complete! Your API key is now protected.")
print("📋 Files created: .gitignore, .env.example, README.md")
print("🚀 Ready for GitHub - your API key will stay private!")

In [ ]:
run_enhanced_chatbot()